# Lecture 9 : Introduction to C Structures

# Part 1 : Primality Testing Revisited

## Consider again our primality tester in C.  In this version we used a pointer to return the divisor as a second argument.

In [ ]:
%%writefile fun_prime_v2.c
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

bool is_prime(long long n, long long* divisor) {
    if (n % 2 == 0) {
        *divisor = 2;
        return false;
    }
    for (long long d = 3; d*d <= n; d+=2) {
        if (n % d == 0) {
            *divisor = d;
            return false;
        }
    }
    return true;
}

int main (int argc, char** argv) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"n");
        return 1; // abnormal exit
    }
    long long n = atoll(argv[1]);
    long long divisor;
    if (is_prime(n,&divisor)) {
        printf ("The number %lld is prime.\n",n);
    } else {
        printf ("The number %lld is not prime since %lld divides it.\n",n,divisor);
    }
}

Writing fun_prime_v2.c


## Here is a version that uses a C structure to return multiple values instead of using a pointer.

## Note the structure intialization syntax on line 12.

In [ ]:
%%writefile fun_prime_v3.c
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

typedef struct {
    bool is_prime;
    long long divisor;
} prime_info;

prime_info is_prime(long long n) {
    prime_info info = { false, 1 };
    if (n % 2 == 0) {
        info.divisor = 2;
        return info;
    }
    for (long long d = 3; d*d <= n; d+=2) {
        if (n % d == 0) {
            info.divisor = d;
            return info;
        }
    }
    info.is_prime = true;
    return info;
}

int main (int argc, char** argv) {
    if (argc < 2) {
        printf ("command usage: %s %s\n",argv[0],"n");
        return 1; // abnormal exit
    }
    long long n = atoll(argv[1]);
    prime_info info = is_prime(n);
    if (info.is_prime) {
        printf ("The number %lld is prime.\n",n);
    } else {
        printf ("The number %lld is not prime since %lld divides it.\n",n,info.divisor);
    }
}

Writing fun_prime_v3.c


In [ ]:
!gcc -o fun_prime_v3 fun_prime_v3.c

In [ ]:
!./fun_prime_v3 5261656080911617

The number 5261656080911617 is prime.


In [ ]:
!./fun_prime_v3 3439315899953761

The number 3439315899953761 is not prime since 58645681 divides it.


# Part 2 : How are C Structures Passed to Functions?

## The following program does not work as expected.  Why not?  

In [ ]:
%%writefile add_v1.c
#include <stdio.h>

typedef struct {
    float x,y;
} vec2;

void vec2_add (vec2 a, vec2 b, vec2 c) {
    c.x = a.x + b.x;
    c.y = a.y + b.y;
}

int main () {
    vec2 u = { 1.0, 2.0 };
    vec2 v = { 2.0, 3.0 };
    vec2 w = { 0, 0 };
    vec2_add(u,v,w);
    printf ("The sum of u and v is (%.2f, %.2f)\n",w.x,w.y);
}

Writing add_v1.c


In [ ]:
!gcc -o add_v1 add_v1.c

In [ ]:
!./add_v1

The sum of u and v is (0.00, 0.00)


## C passes structures to functions **by value** (i.e. by copy)!

## There is an easy way to fix the problem in this case!

In [ ]:
%%writefile add_v2.c
#include <stdio.h>

typedef struct {
    float x,y;
} vec2;

vec2 vec2_add (vec2 a, vec2 b) {
    vec2 c = { a.x + b.x, a.y + b.y };
    return c;
}

int main () {
    vec2 u = { 1.0, 2.0 };
    vec2 v = { 2.0, 3.0 };
    vec2 w = vec2_add(u,v);
    printf ("The sum of u and v is (%.2f, %.2f)\n",w.x,w.y);
}

Writing add_v2.c


In [ ]:
!gcc -o add_v2 add_v2.c

In [ ]:
!./add_v2

The sum of u and v is (3.00, 5.00)


## However there are some pitfalls to keep in mind when passing C structures by value.  

## Discussion: Why does the following code take so long to run?

In [ ]:
%%writefile bigvector_v1.c
#include <stdio.h>

typedef struct {
    float x[200000];
} vec200k;

vec200k vec200k_set (vec200k v, int i, float c) {
    v.x[i] = c;
    return v;
}

int main () {
    vec200k vec;
    for (int i=0;i<200000;i++) {
        vec = vec200k_set (vec,i,i);
    }
    printf ("x[123456] = %.2f",vec.x[123456]);
}

Writing bigvector_v1.c


In [ ]:
!gcc -o bigvector_v1 bigvector_v1.c

In [ ]:
!time ./bigvector_v1

x[123456] = 123456.00
real	0m26.075s
user	0m20.826s
sys	0m0.126s


# Part 3 : Passing C Structures by Pointer

## Like integers, floats, etc., we can pass C structures to function by pointer.

## Here is a third version of our add function that works as expected.  

## In particular, we pass the third argument by pointer.

## Note that c->x is short for (*c).x

In [ ]:
%%writefile add_v3.c
#include <stdio.h>

typedef struct {
    float x,y;
} vec2;

void vec2_add (vec2 a, vec2 b, vec2* c) {
    c->x = a.x + b.x;
    c->y = a.y + b.y;
}

int main () {
    vec2 u = { 1.0, 2.0 };
    vec2 v = { 2.0, 3.0 };
    vec2 w = { 0, 0 };
    vec2_add(u,v,&w);
    printf ("The sum of u and v is (%.2f, %.2f)\n",w.x,w.y);
}

Writing add_v3.c


In [ ]:
!gcc -o add_v3 add_v3.c

In [ ]:
!./add_v3

The sum of u and v is (3.00, 5.00)


## **Passing large C structures by value in C is rarely a good idea.**

## Instead we should always pass large C structures by pointer for efficiency.  

## Here is version 2 of bigvector that is **drastically** more efficient than version 1.

## In fact, if we pass the large C structure by pointer for efficiency, we do not need the return value since we can modify the structure indirectly through the pointer!

In [ ]:
%%writefile bigvector_v2.c
#include <stdio.h>

typedef struct {
    float x[200000];
} vec200k;

void vec200k_set (vec200k* v, int i, float c) {
    v->x[i] = c;
}

int main () {
    vec200k vec;
    for (int i=0;i<200000;i++) {
        vec200k_set (&vec,i,i);
    }
    printf ("x[123456] = %.2f",vec.x[123456]);
}

Writing bigvector_v2.c


In [ ]:
!gcc -o bigvector_v2 bigvector_v2.c

In [ ]:
!time ./bigvector_v2

x[123456] = 123456.00
real	0m0.005s
user	0m0.001s
sys	0m0.002s
